//////////////////////////////   file OBJECT & batch class with test class     ///////////////////////////////////////////////


///////////////////////////////////////////////////////////////////////////////////////   ONE    /////////////////////////////////////////////


In [ ]:
///////////////////////////////////////////////////////////////////////////////////////   trigger    ///////////////////////////////////////////

trigger RelatedDocumentTrigger on Related_Document_History__c (before update , after update) {
    
    if(trigger.isbefore && trigger.isupdate){
        updatepublicLink(trigger.new);
    }
    
    if(trigger.isafter && trigger.isupdate){
         List<Id> rdhids = new List<Id>();
        
        for(Related_Document_History__c p : trigger.new){
            Map<String,ReloyAllowedDocumentType__c> cs = new Map<String,ReloyAllowedDocumentType__c>();
            cs = ReloyAllowedDocumentType__c.getall();
            list<string> alldocuments = new list<string>();
            for(ReloyAllowedDocumentType__c c : cs.values()){
               alldocuments.add(c.Name);
            } 
     
            
            if( Trigger.oldMap.get(p.id).Is_file_uploaded__c != Trigger.newMap.get(p.id).Is_file_uploaded__c && Trigger.newMap.get(p.id).Is_file_uploaded__c== true && alldocuments.contains(p.Document_Type__c) &&  string.isNotBlank(p.Document_Type__c)){
               system.debug('workingblank'+cs);
            rdhids.add(p.id);
            system.enqueueJob(new EX_DocumentReloyApplication(rdhids));
            }
        }
        
    }
    
   //  updatepubliclink method    
    public static void updatepublicLink(list<Related_Document_History__c> nw){
        system.debug('updatepublicLink');
         Id relatedId;
    String demandDocLink = '';
    for(Related_Document_History__c pl : nw){
        
        if(pl.Is_file_uploaded__c == true && pl.ispublicLinkCreated__c==false){
             relatedId = pl.Id;
         List<ContentDocumentLink> cdlinkList = new List<ContentDocumentLink>();
               cdlinkList = [SELECT Id, LinkedEntityId, ContentDocumentId,ContentDocument.LatestPublishedVersionId,Visibility, IsDeleted, ShareType,ContentDocument.Title,ContentDocument.createdDate, ContentDocument.FileType FROM ContentDocumentLink WHERE LinkedEntityId =:relatedId Order By ContentDocument.createdDate DESC limit 1];
    system.debug(cdlinkList);
        
         for(ContentDocumentLink link:cdlinkList){
                    
                    ContentDistribution cd = new ContentDistribution();
                    cd.Name = link.ContentDocument.Title;
                    cd.ContentVersionId = link.ContentDocument.LatestPublishedVersionId;
                    cd.PreferencesAllowViewInBrowser= true;
                    cd.PreferencesLinkLatestVersion=true;
                    cd.PreferencesNotifyOnVisit=false;
                    cd.PreferencesPasswordRequired=false;
                    cd.PreferencesAllowOriginalDownload= true;
                    insert cd;   
                    cd = [SELECT Id,DistributionPublicUrl FROM ContentDistribution WHERE Id = :cd.Id LIMIT 1];
                    if(String.isNotBlank(cd.DistributionPublicUrl)){
                      system.debug(cd.DistributionPublicUrl);
                       system.debug(cd.DistributionPublicUrl); 
                        system.debug('DistributionPublicUrl::'+cd.DistributionPublicUrl);
                       demandDocLink = cd.DistributionPublicUrl;
                    } 
                  
                }
        
      pl.AttachmentIdPublicLink__c = demandDocLink;
      pl.ispublicLinkCreated__c = true;     
        }
       
   
    }
    }
}

///////////////////////////////////////////////////////////////////////////////////////   class    ///////////////////////////////////////////

//TEST CLASS - TEST_EX_DocumentReloyApplication
public class EX_DocumentReloyApplication implements Queueable, Database.AllowsCallouts {
public List<Id> appIds = new List<Id>();
     public EX_DocumentReloyApplication(List<Id> documentIds){
        appIds.addAll(documentIds);
    }
    public void execute(QueueableContext context) {
         String bookingCrmId='';
        String customerCrmId='';
        String documentNumber='';
        String folder='';
        String link='';
        String projectCrmId='';
        String state='';
        String type='';
        String unitNumber='';
        
         String jsonS = '';
        
          Http http = new Http();
        HTTPResponse response;
        List<System_Log__c> listSysLog = new List<System_Log__c>();
        List<Related_Document_History__c> listAppDetails = new List<Related_Document_History__c>();
        listAppDetails = [SELECT Id ,Document_Type__c, Name,   AttachmentIdPublicLink__c ,Booking__r.Name , Booking__r.Unit__c ,Booking__r.Project__c ,Booking__r.Opportunity__r.AccountId FROM Related_Document_History__c where Id =: appIds];
        system.debug('listAppDetails'+listAppDetails);
        system.debug('id'+appIds);
        try{
            if(!listAppDetails.isEmpty()){
                link = listAppDetails[0].AttachmentIdPublicLink__c;
                bookingCrmId = listAppDetails[0].Booking__r.Name;  
                customerCrmId = listAppDetails[0].Booking__r.Opportunity__r.AccountId;  
                documentNumber = listAppDetails[0].Name;  
                folder = '';  
                projectCrmId = listAppDetails[0].Booking__r.Project__c;  
                 state = '';  
                 type = listAppDetails[0].Document_Type__c;  
                unitNumber = listAppDetails[0].Booking__r.Unit__c;  
                
                  JSONGenerator gen = JSON.createGenerator(true);
            gen.writeStartArray();
            gen.writeStartObject();
                
                  if(String.isNotBlank(bookingCrmId))
            {
            gen.writeStringField('bookingCrmId',bookingCrmId);
            }
                    if(String.isNotBlank(customerCrmId))
            {
            gen.writeStringField('customerCrmId',customerCrmId);
            }
                    if(String.isNotBlank(documentNumber))
            {
            gen.writeStringField('documentNumber',documentNumber);
            }
                    if(String.isNotBlank(folder))
            {
            gen.writeStringField('folder',folder);
            }
                    if(String.isNotBlank(link))
            {
            gen.writeStringField('link',link);
            }
                      if(String.isNotBlank(projectCrmId))
            {
            gen.writeStringField('projectCrmId',projectCrmId);
            }
                          if(String.isNotBlank(state))
            {
            gen.writeStringField('state',state);
            }
            //if(String.isNotBlank(type))
            //{type
            gen.writeStringField('type','OTHER');
            //}
                                if(String.isNotBlank(unitNumber))
            {
            gen.writeStringField('unitNumber',unitNumber);
            }
                   gen.writeEndObject();   
            gen.writeEndArray();
            jsonS = gen.getAsString();
            System.debug('jsonMaterials'+jsonS);
                
                 String endpoint = 'https://uat.loyalie.in/winnre/crmApi/documentUpsert';
           HttpRequest req = new HttpRequest();
            req.setEndpoint(endpoint);
            req.setHeader('Content-Type', 'application/json');
            req.setHeader('Accept','*/*');
            req.setHeader('Client-Id','51808609e781b8dd8ba561acf88914887185adba2fe5a2698192e9d19d24dc53');
            req.setHeader('requestKey','VB0yl2x/CyUiljq3BhadUg==');
            req.setMethod('POST');
            req.setbody(jsonS);
            system.debug('request: '+ req);
            system.debug('JSON Body: '+ jsonS);
            
            if(!test.isRunningTest())
            {
                 response=http.send(req);
            }
            system.debug('response: '+ response);
            System_Log__c objSysLog = new System_Log__c();
            objSysLog.Action__c='DocumentUpsert Api';
            objSysLog.Object_Name__c='Related Document History';
            objSysLog.Opportunity__c=listAppDetails[0].Booking__r.Opportunity__c;
            objSysLog.Remarks__c='Document API Details--> Request:: '+jsonS+' -Response:: '+response;
            listSysLog.add(objSysLog);
            }
            
        }catch(Exception e){
            system.debug('Exception:'+e);
             System_Log__c objSysLog = new System_Log__c();
            objSysLog.Action__c='DocumentUpsert Api';
            objSysLog.Object_Name__c='Related Document History';
           // objSysLog.Opportunity__c=listAppDetails[0].Booking__r.Opportunity__c;
            objSysLog.Remarks__c='Document API Details--> Request:: '+jsonS+' -Response:: '+response+' Exception = '+E;
            listSysLog.add(objSysLog);
        }
         if(!listSysLog.isEmpty())
            {
                Database.insert(listSysLog,false);
                 
            } 
          
    }
}

///////////////////////////////////////////////////////////////////////////////////////   test class     ///////////////////////////////////////////


@istest
public class TEST_EX_DocumentReloyApplication {
    @istest
    public static void TEST_EX_DocumentReloyApplicationmethod(){
         
         ReloyAllowedDocumentType__c cs = new ReloyAllowedDocumentType__c();
         cs.Name='Registered AFS';
         insert cs;
        
        Project__c p = new Project__c();
        p.Name = 'new project';
        p.Project_Code__c = '011sb';
        insert p;
        
         Opportunity opp2     = new Opportunity();
        opp2.Name        = 'test opp2';
        opp2.CloseDate   = system.today()+1;
        opp2.StageName   = 'Initial Pitch/Demo';
        opp2.Project__c = P.Id;
        insert opp2;
        
        Unit__c un = new Unit__c();
        un.Name = '205';
        un.Project__c=p.Id;
        un.Project_Property_Type__c = 'Residential';
        insert un;
        
        Booking__c BK = NEW Booking__c();
        BK.Booking_Date__c = system.today();
        BK.Unit__c = un.Id;
        INSERT BK;
        
        
        Related_Document_History__c RH = NEW Related_Document_History__c();
        RH.Document_Name__c = 'Registered AFS';
        RH.Document_Type__c = 'Registered AFS';
        RH.Booking__c = BK.Id;
        INSERT RH;
   
  
       
        
        System_Log__c sy = new System_Log__c();
        sy.Action__c = 'CustomerUpsert Api';
        insert sy;
        
           //Create a new file
        ContentVersion newFile = new ContentVersion();
        newFile.PathOnClient = 'TestFile';
        newFile.Title = 'TestFile';
        newFile.VersionData = Blob.valueOf('sample content');
        insert newFile;
        
     
        
        //Query the contentDocumentId
        ContentVersion insertedFile = [SELECT Id, ContentDocumentId FROM ContentVersion WHERE Id =:newFile.Id];
        
        //Link the file with the product
        ContentDocumentLink linkedFile = new ContentDocumentLink(LinkedEntityId=RH.Id, ContentDocumentId=insertedFile.ContentDocumentId);
        insert linkedFile;
 
         
        RH.Is_file_uploaded__c = true;
        RH.AttachmentIdPublicLink__c = 'TestLink';
        update RH;
    }
}


///////////////////////////////////////////////////////////////////////////////////////   TWO    /////////////////////////////////////////////


In [ ]:
///////////////////////////////     BATCH CLASS

global class ReceiptPublicLinkUpdateBatch implements Database.Batchable <sObject>, Database.AllowsCallouts {
    //static boolean recursive = false;

    global Database.QueryLocator start(Database.BatchableContext BC) {
       
        String query = '';
        Boolean bVal = false;
   /*     if(Test.isRunningTest()){
         query = 'Select Id, Name, Amount_Rs__c, Mode__c, Receipt_Date__c,Date_Of_Apportionment__c,Cheque_DD_Date__c, DraweeBank__c,Drawee_Bank_If_Others__c,Tax_Receipt__c,Opportunity__c,Receipt_Number__c,' 
         +'Drawn_in_favour_of__c, Cheque_DD__c, On_Account_Money__c, On_Account_Money_Autocalculated__c, Receipt_status__c,RemarksText__c,Booking__c,Advance_Receipt__c,'
         +' Amount_in_Foreign_Currency__c,PAN_no_of_TDS_Deductor__c, PAN_no_of_TDS_Deductee__c, Challan_No__c,TDS_Amount_In_Rs__c,IsTDS_And_NoDemandRaised__c,'    
         +'Currency__c, FOREX__c, Project__c,Project__r.Id,AttachmentIdPublicLink__c,Project_Unit__c,Approval_Status__c,Opportunity_Email_Id__c,ispublicLinkCreated__c from receipt__c where ispublicLinkCreated__c ='+bVal+' order by CreatedDate DESC limit 1';
   *///      }else{
         query = 'Select Id, Name, Amount_Rs__c, Mode__c, Receipt_Date__c,Date_Of_Apportionment__c,Cheque_DD_Date__c, DraweeBank__c,Drawee_Bank_If_Others__c,Tax_Receipt__c,Opportunity__c,Receipt_Number__c,' 
         +'Drawn_in_favour_of__c, Cheque_DD__c, On_Account_Money__c, On_Account_Money_Autocalculated__c, Receipt_status__c,RemarksText__c,Booking__c,Advance_Receipt__c,'
         +' Amount_in_Foreign_Currency__c,PAN_no_of_TDS_Deductor__c,AttachmentIdPublicLink__c, PAN_no_of_TDS_Deductee__c, Challan_No__c,TDS_Amount_In_Rs__c,Print_Receipt_Text__c,IsTDS_And_NoDemandRaised__c,'    
         +'Currency__c, FOREX__c, Project__c,Project__r.Id,Project_Unit__c,Approval_Status__c,Opportunity_Email_Id__c,ispublicLinkCreated__c from receipt__c where ispublicLinkCreated__c ='+bVal+' order by CreatedDate DESC limit 1';
   //     } 
                
        return Database.getQueryLocator(query);
    }
    
    global void execute(Database.BatchableContext BC, List < Receipt__c > scope) {
        
        List<Receipt__c> rListToUpdate = new List<Receipt__c>();
        Id relatedId=null;
        String demandDocLink = '';
        
        try{
            for(Receipt__c r:scope){
                relatedId = r.id;
                List<ContentDocumentLink> cdlinkList = new List<ContentDocumentLink>();
                List<ContentVersion> contentVersionList = new List<ContentVersion>();
                
                system.debug('A'+r.Print_Receipt_Text__c);
                
              if(String.isNotBlank(r.AttachmentIdPublicLink__c)){
                    //receiptLink = r.AttachmentIdPublicLink__c; 
                     r.ispublicLinkCreated__c=true;
                     rListToUpdate.add(r);
             }else{        
                
                if(String.isNotBlank(r.Print_Receipt_Text__c)&&r.Print_Receipt_Text__c!='NOT Available'){
                        system.debug('printPage:'+r.Print_Receipt_Text__c);
                  
                 
                      PageReference pageRef = new PageReference(r.Print_Receipt_Text__c); 
                   
                        BLOB body;
                   
                        try{  
                    if(!test.isRunningTest()) {                   //Commented on 29/12/2022
                                body = pageRef.getContentAsPDF();
                        }else{
                            body = BLOB.valueOf('Test');
                         }
                            
                        }catch (VisualforceException e) {
                            system.debug('in the catch block');
                            body = Blob.valueOf('Some Text');
                        }
                        system.debug('body:'+body);
                        ContentVersion cont = generateNewContentVersion(body, r);
                        //createFeedItem(cont.Id, rList);
                        
                        ContentVersion cv = [select id,Title from contentversion where Id=:cont.Id];
                        String Title  = cv.Title;
                        if(Title.length()>80){
                            Title = Title.substring(0,80);  
                        }
                        ContentDistribution cd = new ContentDistribution();
                        cd.Name = Title;
                        cd.ContentVersionId = cv.id;
                        cd.PreferencesAllowViewInBrowser= true;
                        cd.PreferencesLinkLatestVersion=true;
                        cd.PreferencesNotifyOnVisit=false;
                        cd.PreferencesPasswordRequired=false;
                        cd.PreferencesAllowOriginalDownload= true;
                        insert cd;   
                        cd = [SELECT Id,DistributionPublicUrl FROM ContentDistribution WHERE Id = :cd.Id LIMIT 1];
                        system.debug('cd:'+cd);
                        system.debug('preview:'+cd.DistributionPublicUrl);
                        r.AttachmentIdPublicLink__c = cd.DistributionPublicUrl;
                        r.ispublicLinkCreated__c=true;
                        //receiptLink = cd.DistributionPublicUrl;
                          rListToUpdate.add(r);
                    
                }  
             }
                
            }
            if(!rListToUpdate.isEmpty()){
                update rListToUpdate;
            }  
            
        }catch(Exception e){
            system.debug('Exception:'+e);
            system.debug('ExceptionLine:'+e.getLineNumber());
        }
    }
    
    global void finish(Database.BatchableContext BC) {
        // if(!recursive) {
       // recursive = true;
        // Do your stuff here
       //}
         AsyncApexJob a = [Select Id, Status,ExtendedStatus,NumberOfErrors,     JobItemsProcessed,
        TotalJobItems, CreatedBy.Email
        from AsyncApexJob where Id =:BC.getJobId()];
          
            Messaging.SingleEmailMessage mail = new Messaging.SingleEmailMessage();
            String[] toAddresses = new String[] {'yogesh.swami@excellerconsultancy.in'};
            mail.setToAddresses(toAddresses);
            mail.setSubject('Receipt PublicLink Batch Completed' + a.Status);
            mail.setPlainTextBody('records processed ' + a.TotalJobItems +
           'with '+ a.NumberOfErrors + ' failures.');
            Messaging.sendEmail(new Messaging.SingleEmailMessage[] { mail });
    }
    global static ContentVersion generateNewContentVersion(Blob fileContent, Receipt__c r) {
        ContentVersion cont = new ContentVersion();
        String Title  = r.name;
        system.debug('Title::::'+Title);
        if(Title!=null){
            if(Title.length()>80){
                Title = Title.substring(0,80);  
            }
        }
        
        cont.Title =  Title;
        //cont.Title = invNo + '-' + d.Booking__r.Opportunity__r.Name + '-' + d.Booking__r.Opportunity__r.Unit__r.Tower__r.Cluster__r.Name + '-' + d.Booking__r.Opportunity__r.Unit__r.Tower__r.Name + '-' + d.Booking__r.Opportunity__r.Unit__r.Name + system.now();
        system.debug('fileContent::::'+fileContent);
        
        cont.VersionData = fileContent;
        system.debug('fileContent::::'+fileContent);
        
        cont.PathOnClient =  r.name +'.pdf';
        cont.ContentLocation = 'S';
        cont.Description = 'Receipt for :' +  r.name;
        date dt = date.newInstance(system.today().Year(),system.today().Month(),system.today().Day());
        // cont.TagCSV =  r.Opportunity__c;
        insert cont;
        return cont;
    }

}

////////////////////////////////////////////// TEST CLASS

@istest(SeeAllData=true)
public class TEST_ReceiptPublicLinkUpdateBatch {
  @istest
    public static void TEST_ReceiptPublicLinkUpdateBatchmethod(){
        
        
           Project__c p = new Project__c();
        p.Name = 'new project';
        p.Project_Code__c = '011sb';
        insert p;
        
        Tower__c T = new Tower__c();
        T.Project__c = p.Id;
        T.Project_Due__c = 50;
        insert T;
        
         Opportunity opp2     = new Opportunity();
        opp2.Name        = 'test opp2';
        opp2.CloseDate   = system.today()+1;
        opp2.StageName   = 'Initial Pitch/Demo';
        opp2.Project__c = P.Id;
        opp2.Tower__c = T.Id;
        insert opp2;
        
        Unit__c un = new Unit__c();
        un.Name = '205';
        un.Project__c=p.Id;
        un.Project_Property_Type__c = 'Residential';
        un.Tower__c = T.Id;
        insert un;
        
        Booking__c BK = NEW Booking__c();
        BK.Booking_Date__c = system.today();
        BK.Unit__c = un.Id;
        INSERT BK;
        
         Receipt__c a = [SELECT Id, Name , Print_Receipt_Text__c , Mode__c FROM Receipt__c WHERE Name = 'R-15773'];
          list<Receipt__c> rclist = new list<Receipt__c>();
        
        Receipt__c RP = new Receipt__c();
        RP.Project__c = p.Id;
        RP.Project_Unit__c = un.Id;
        RP.Opportunity__c = opp2.Id;
        RP.Mode__c = 'EFT';
        RP.Cheque_DD_Date__c = system.today();
        RP.Receipt_Cumulative__c = 6;
        RP.Amount_Rs__c = 10000;
        RP.Total_Amount__c = 10000;
        RP.AttachmentIdPublicLink__c = '';
      
      
       //  PageReference pageRef = new PageReference(RP.Mode__c);
    //    RP.Print_Receipt_Text__c = 'Credit Note';
        rclist.add(RP);
        insert rclist;
        
        
        //Create a new file
        ContentVersion newFile = new ContentVersion();
        newFile.PathOnClient = 'TestFile';
        newFile.Title = 'TestFile';
        newFile.VersionData = Blob.valueOf('sample content');
        insert newFile;
        
     
        
        //Query the contentDocumentId
        ContentVersion insertedFile = [SELECT Id, ContentDocumentId FROM ContentVersion WHERE Id =:newFile.Id];
        
        //Link the file with the product
        ContentDocumentLink linkedFile = new ContentDocumentLink(LinkedEntityId=RP.Id, ContentDocumentId=insertedFile.ContentDocumentId);
        insert linkedFile;
        
        ///
    
       
        //
           test.starttest();
        database.executeBatch(new ReceiptPublicLinkUpdateBatch(),1);
        test.stoptest();
        
        //   generateNewContentVersion
        String s = 'abc';
Blob b = Blob.valueOf(s);
        
         ContentVersion cont = new ContentVersion();
         cont.Title =  'Title';
         cont.ContentLocation = 'S';
        
        ReceiptPublicLinkUpdateBatch.generateNewContentVersion(b,  RP);
       //
    }
}



///////////////////////////////////////////////////////////////////////////////////////   THREE    /////////////////////////////////////////////


In [ ]:
///////////////////////////////     BATCH CLASS

global class DemandPublicLinkUpdateBatch implements Database.Batchable <sObject>, Database.AllowsCallouts {
    //static boolean recursive = false;

    global Database.QueryLocator start(Database.BatchableContext BC) {
        String query = '';
        Boolean bVal = false;
        if(Test.isRunningTest()){
         query = 'SELECT Id, Name, Project_Name_For_Demands__c ,Invoice_Date__c, Due_Date__c,Booking__r.Primary_Applicant_Name__c,Milestone_Display_Name__c, Invoice_Number__c, Booking__r.Opportunity__r.Name,' 
         +'Booking__r.Opportunity__r.Unit__r.Id,ispublicLinkCreated__c,Booking__r.Opportunity__r.Unit__r.Name, Booking__r.Opportunity__r.Unit__r.Tower__r.Name,Invoice_Number_F__c,Don_t_Generate_Sequence_No__c,'
         +'Booking__r.Opportunity__r.Unit__r.Tower__r.Cluster__r.Name,AttachmentId__c,AttachmentId_With_Header__c,AttachmentIdPublicLink__c,Booking__r.Opportunity__r.Customer_Number__c From Demand__c WHERE ispublicLinkCreated__c ='+bVal+' order by CreatedDate DESC limit 1';
        }else{
         query = 'SELECT Id, Name, Project_Name_For_Demands__c ,Invoice_Date__c, Due_Date__c,Booking__r.Primary_Applicant_Name__c,Milestone_Display_Name__c, Invoice_Number__c, Booking__r.Opportunity__r.Name,' 
         +'Booking__r.Opportunity__r.Unit__r.Id,ispublicLinkCreated__c,Booking__r.Opportunity__r.Unit__r.Name, Booking__r.Opportunity__r.Unit__r.Tower__r.Name,Invoice_Number_F__c,Don_t_Generate_Sequence_No__c,'
         +'Booking__r.Opportunity__r.Unit__r.Tower__r.Cluster__r.Name,AttachmentId__c,AttachmentId_With_Header__c,AttachmentIdPublicLink__c,Booking__r.Opportunity__r.Customer_Number__c From Demand__c WHERE ispublicLinkCreated__c ='+bVal+' order by CreatedDate DESC limit 1';
        } 
        return Database.getQueryLocator(query);
    }
    
    global void execute(Database.BatchableContext BC, List < Demand__c > scope) {
        
        List<Demand__c> dlistToUpdate = new List<Demand__c>();
        Id relatedId=null;
        String demandDocLink = '';
        try{
            for(Demand__c d:scope){
                relatedId = d.id;
                List<ContentDocumentLink> cdlinkList = new List<ContentDocumentLink>();
                List<ContentVersion> contentVersionList = new List<ContentVersion>();
                
                cdlinkList = [SELECT Id, LinkedEntityId, ContentDocumentId,ContentDocument.LatestPublishedVersionId,Visibility, IsDeleted, ShareType,ContentDocument.Title,ContentDocument.createdDate, ContentDocument.FileType FROM ContentDocumentLink WHERE LinkedEntityId =:relatedId Order By ContentDocument.createdDate DESC limit 1];
                system.debug('cdl=' + cdlinkList);
                for(ContentDocumentLink link:cdlinkList){
                    
                    ContentDistribution cd = new ContentDistribution();
                    String title = link.ContentDocument.Title;
                    if(String.isNotBlank(title)){
                    if(title.length()>80){
                        cd.Name = title.substring(0,70); 
                    }else{
                        cd.Name = title;  
                    }
                    }else{
                      cd.Name = 'Doc-'+System.now(); 
                    }
                    cd.ContentVersionId = link.ContentDocument.LatestPublishedVersionId;
                    cd.PreferencesAllowViewInBrowser= true;
                    cd.PreferencesLinkLatestVersion=true;
                    cd.PreferencesNotifyOnVisit=false;
                    cd.PreferencesPasswordRequired=false;
                    cd.PreferencesAllowOriginalDownload= true;
                    insert cd;   
                    cd = [SELECT Id,DistributionPublicUrl FROM ContentDistribution WHERE Id = :cd.Id LIMIT 1];
                    if(String.isNotBlank(cd.DistributionPublicUrl)){
                        d.AttachmentIdPublicLink__c = cd.DistributionPublicUrl;
                        d.ispublicLinkCreated__c = true;
                        system.debug('DistributionPublicUrl::'+cd.DistributionPublicUrl);
                    } 
                    //
                }  
                dlistToUpdate.add(d);
            }
            if(!dlistToUpdate.isEmpty()){
                update dlistToUpdate;
            }  
            
        }catch(Exception e){
            system.debug('Exception:'+e);
            system.debug('ExceptionLine:'+e.getLineNumber());
        }
    }
    
    global void finish(Database.BatchableContext BC) {
        // if(!recursive) {
       // recursive = true;
        // Do your stuff here
    //}
      AsyncApexJob a = [Select Id, Status,ExtendedStatus,NumberOfErrors,     JobItemsProcessed,
        TotalJobItems, CreatedBy.Email
        from AsyncApexJob where Id =:BC.getJobId()];
          
            Messaging.SingleEmailMessage mail = new Messaging.SingleEmailMessage();
            String[] toAddresses = new String[] {'yogesh.swami@excellerconsultancy.in'};
            mail.setToAddresses(toAddresses);
            mail.setSubject('Documents PublicLink Batch Completed' + a.Status);
            mail.setPlainTextBody('records processed ' + a.TotalJobItems +
           'with '+ a.NumberOfErrors + ' failures.');
            Messaging.sendEmail(new Messaging.SingleEmailMessage[] { mail });
    }
}



///////////////////////////////     TEST CLASS

@istest
public class TEST_DemandPublicLinkUpdateBatch {
      @istest
    public static void TEST_DemandPublicLinkUpdateBatchmethod(){
          Project__c p = new Project__c();
        p.Name = 'new project';
        p.Project_Code__c = '011sb';
        insert p;
        
         Opportunity opp2     = new Opportunity();
        opp2.Name        = 'test opp2';
        opp2.CloseDate   = system.today()+1;
        opp2.StageName   = 'Initial Pitch/Demo';
        opp2.Project__c = P.Id;
        insert opp2;
        
        Unit__c un = new Unit__c();
        un.Name = '205';
        un.Project__c=p.Id;
        un.Project_Property_Type__c = 'Residential';
        insert un;
        
        Booking__c BK = NEW Booking__c();
        BK.Booking_Date__c = system.today();
        BK.Unit__c = un.Id;
        INSERT BK;
        
      list<Demand__c> dclist = new list<Demand__c>();
        
          Demand__c dc = new Demand__c();
        dc.Invoice_Date__c = system.today();
        dc.Booking__c = BK.Id;
        dclist.add(dc);
        insert dclist;
        
        //Create a new file
        ContentVersion newFile = new ContentVersion();
        newFile.PathOnClient = 'TestFile';
        newFile.Title = 'TestFile';
        newFile.VersionData = Blob.valueOf('sample content');
        insert newFile;
        
     
        
        //Query the contentDocumentId
        ContentVersion insertedFile = [SELECT Id, ContentDocumentId FROM ContentVersion WHERE Id =:newFile.Id];
        
        //Link the file with the product
        ContentDocumentLink linkedFile = new ContentDocumentLink(LinkedEntityId=dc.Id, ContentDocumentId=insertedFile.ContentDocumentId);
        insert linkedFile;
        
      
     test.starttest();
        database.executeBatch(new DemandPublicLinkUpdateBatch(),1);
        test.stoptest();
    }
}
